In [1]:
#https://python.langchain.com/v0.1/docs/use_cases/sql/quickstart/

In [6]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [9]:
from langchain.chains import create_sql_query_chain
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.1:latest",temperature=0)
chain = create_sql_query_chain(llm, db)
#response = chain.invoke({"question": "How many customers are there"})
#response

'SELECT COUNT(CustomerId) FROM Customer'

'[(59,)]'

In [ ]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.agents import AgentExecutor, Tool

# Define the prompt template for human-readable output
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

# Define the SQL chain
sql_chain = (
    RunnablePassthrough.assign(query=write_query)  # Generate the SQL query
    .assign(result=itemgetter("query") | execute_query)  # Execute the query and return the result
    | answer_prompt  # Fill in the prompt with question, query, and result
    | llm  # Use the LLM to generate a human-readable answer
    | StrOutputParser()  # Parse the output as a string
)

# Define a tool that the agent can use (our SQL chain tool)
sql_tool = Tool(
    name="SQLQueryTool",
    func=sql_chain.invoke,  # The chain's invoke function
    description="Executes a SQL query based on a user question and returns a natural language answer."
)

# Define the agent that will handle user input
agent = AgentExecutor.from_tools([sql_tool])

# Now you can invoke the agent directly, and it will use the chain as one of its tools
response = agent.invoke({"input": "How many employees are there?"})

# Print the final response
print(response)


Query checerk

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [13]:
query = full_chain.invoke({"question": "How many employees are there"})

In [14]:
query

'SELECT * FROM customers;'

In [15]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result